In [ ]:
%matplotlib inline

In [ ]:
import xarray as xr
import matplotlib.pyplot as plt

from lib.models.torch import wrap
import torch
import holoviews as hv
hv.extension('matplotlib')

plt.rcParams['savefig.dpi'] = 125

Some utility plotting functions.

In [ ]:
def prog_plot(phi, autolevels=False, figsize=(8, 4)):
    
    if not autolevels:
        qt_kw = dict(levels=np.arange(11) * 2.5, extend='both')
        t_kw = dict(levels=np.arange(12)*5 + 275, extend='both')
    else:
        qt_kw = {}
        t_kw = {}

    fig, axs = plt.subplots(2, 1, sharex=True, sharey=True, figsize=figsize, facecolor='w',
                           dpi=100)
    
    t_im = axs[0].contourf(phi.time, phi.p, phi.qt.T, **qt_kw)
    q_im = axs[1].contourf(phi.time, phi.p, phi.sl.T, **t_kw)

    plt.colorbar(t_im, ax=axs[0])
    plt.colorbar(q_im, ax=axs[1])

    axs[0].set_ylabel('sl')
    axs[1].set_ylabel('qt')    
    axs[0].invert_yaxis()
    
    for ax in axs:
        ax.set_ylabel('p [hPa]')
    
    axs[-1].set_xlabel('days')
    
    
    bbox = dict(color='w')
    axs[1].text(105, 250, r"$s_l$ [K]", bbox=bbox)
    axs[0].text(105, 250, r"$q_T$ [g/kg]", bbox=bbox)        
    return axs

In [ ]:
phi = xr.open_dataset("../../data/processed/inputs.nc")
g = xr.open_dataset("../../data/processed/forcings.nc")
mod = torch.load("../../data/ml/ngaqua/model.1.torch")

phi_loc = phi.isel(x=0, y=8)
g_loc = g.isel(x=0, y=8)

# Slide 12

This is the truth

In [ ]:
prog_plot(phi_loc);

This is the forcing

In [ ]:
prog_plot(g_loc.assign(p=phi.p), autolevels=True);

This is the unforced example. this shows the simulation has a stable RCE

In [ ]:
pred_unforced = wrap(mod)(phi_loc, 0*g_loc).assign(p=phi.p)
prog_plot(pred_unforced);

In [ ]:
pred_forced = wrap(mod)(phi_loc, g_loc).assign(p=phi.p)
prog_plot(pred_forced);

In [ ]:
phi_loc

# Precipitation

In [ ]:
from lib.thermo import precip_from_ds, precip_from_dq, mse, mass_integrate

In [ ]:
fields_2d = xr.open_dataset("../../data/raw/2/NG_5120x2560x34_4km_10s_QOBS_EQX/coarse/2d/all.nc")
fields_3d = xr.open_dataset("../../data/raw/2/NG_5120x2560x34_4km_10s_QOBS_EQX/coarse/3d/QRAD.nc")\
              .drop('p')
data = xr.merge((phi, fields_2d, fields_3d), join='inner')

In [ ]:
phi = phi.transpose('time', 'x', 'y', 'z')

In [ ]:
Q = wrap(mod.step.rhs)(phi)

In [ ]:
prect = precip_from_ds(Q.sl, data.QRAD, data.SHF, data.p)
precq = precip_from_dq(Q.qt, data.LHF, data.p)
prec = xr.Dataset(dict(prect=prect, precq=precq, prec=data.Prec))

In [ ]:
def loc_plot(prec):
    plt.figure(figsize=(8, 4))
    prec.prec.plot(label='Truth', c='k')
    prec.prect.plot(label='Precip from Q1', c='b')
    prec.precq.plot(label='Precip from Q2', c='g')
    plt.ylabel('mm/day')
    plt.legend()
    plt.xlim([100,150])
    
loc_plot(prec.isel(x=0, y=8))

Why does it looks like there is a diurnal cycle in the Q1 derived precipitation?

In [ ]:
plt.figure(figsize=(6,3), dpi=120, facecolor='w')
q1_vert_int = mass_integrate(phi.p, Q.sl) * 1004/2.5e6
q1_vert_int.isel(x=0,y=8).plot(label=r'$\langle Q_{1} \rangle$')
plt.legend()

Because this mass weighted vertical intergral of $Q_1$ does not show this diurnal behaviour, this is clearly a function of the convective scheme not accurately capturing the diurnal cycle of precipitation correctly.

In [ ]:
%%output size=100
%%opts Scatter(alpha=.25)

def hmap_loc(prec):
    hmap = hv.HoloMap({
        'Prec from Q2': hv.Scatter((prec.precq, prec.prec)),
        'Prec from Q1': hv.Scatter((prec.prect, prec.prec))
    })
    
    return hmap

hmap = hmap_loc(prec.isel(x=0, y=8))
hmap.layout().cols(1).redim.label(x="Prediction", y="Truth")

In [ ]:
prec.mean(['x', 'time']).to_dataframe().plot()